# model

In [6]:
# Cell 1: Import libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine

host = "localhost"
user = "root"
password = "hello"
database = "fitness"

try:
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
    query = "SELECT * FROM exercise"
    df = pd.read_sql(query, engine)
    df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    print("✅ Successfully connected and retrieved data!")
except Exception as e:
    print("Error:", e)
    df = None

if df is not None:
    df["combined_features"] = (
        df["type"].fillna("") + " " + 
        df["difficulty"].fillna("") + " " + 
        df["equipment"].fillna("") + " " + 
        df["muscle"].fillna("") + " " + 
        df["category"].fillna("")
    )
    
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    
    vectorizer = TfidfVectorizer(stop_words='english')
    feature_matrix = vectorizer.fit_transform(train_df["combined_features"])
    
    print("Data preparation complete!")

    import pickle

# Save vectorizer and model
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))
df.to_pickle('model.pkl')



✅ Successfully connected and retrieved data!
Data preparation complete!


# Testing

In [4]:

# Cell 4: Define user details and preferences
# Example user details
user_details = {
    "age": 28,
    "gender": "male",
    "height": 180,  # cm
    "weight": 150   # kg
}

# Example user preferences with multiple muscle groups
user_preferences = {
    "goal": "lose weight",
    "experience": "advance",
    "equipment": "machine",
    "muscle": "back, biceps"  # Multiple muscle groups
}

# Cell 5: Calculate workout size
# Base number of exercises
base_count = 5

# Adjust based on experience level
experience_modifier = {
    'beginner': -1,
    'intermediate': 0,
    'advanced': 1
}

# Adjust based on goal
goal_modifier = {
    'lose weight': 1,  # More exercises for cardio/weight loss
    'build muscle': 0,
    'strength': -1,    # Fewer but more intense exercises for strength
    'endurance': 2     # More exercises for endurance
}

# Get modifiers (with defaults if keys don't exist)
exp_mod = experience_modifier.get(user_preferences.get('experience', 'intermediate'), 0)
goal_mod = goal_modifier.get(user_preferences.get('goal', 'build muscle'), 0)

# Calculate total exercises
total_exercises = base_count + exp_mod + goal_mod

# Adjust for number of muscle groups
muscles = user_preferences.get('muscle', '').split(',')
num_muscles = len(muscles)

# Ensure at least 1 exercise per muscle group
workout_size = max(total_exercises, num_muscles)

print(f"Recommended workout size: {workout_size} exercises")

# Cell 6: Generate recommendations
# Handle multiple muscle groups
muscles = user_preferences.get('muscle', '').split(',')

# Create a list to store all recommendations
all_recommendations = []

# Get recommendations for each muscle group
for muscle in muscles:
    muscle = muscle.strip()
    
    # Adjust user preferences for this specific muscle
    current_preferences = user_preferences.copy()
    current_preferences['muscle'] = muscle
    
    # Create user query for this muscle
    user_query = (
        f"{current_preferences.get('goal', '')} "
        f"{current_preferences.get('experience', '')} "
        f"{current_preferences.get('equipment', '')} "
        f"{muscle}"
    )
    
    # Transform user query into vector space
    user_vector = vectorizer.transform([user_query])
    
    # Calculate similarities
    similarities = cosine_similarity(user_vector, feature_matrix)
    
    # Add similarity scores to training dataframe (temporary)
    temp_df = train_df.copy()
    temp_df["similarity_score"] = similarities[0]
    
    # Filter by specific muscle if provided
    if muscle:
        # Partial matching for muscle to handle cases like "chest" matching "upper chest"
        muscle_filtered = temp_df[temp_df['muscle'].str.contains(muscle, na=False)]
        
        # If no exercises found for this muscle, use original recommendations
        if not muscle_filtered.empty:
            temp_df = muscle_filtered
    
    # Get top recommendations for this muscle
    muscle_recommendations = temp_df.sort_values(
        by="similarity_score", ascending=False
    ).head(max(3, workout_size // len(muscles)))
    
    # Add to our overall recommendations
    all_recommendations.append(muscle_recommendations)

# Cell 7: Combine and display final workout plan
if all_recommendations:
    recommendations = pd.concat(all_recommendations)
    
    # Remove duplicates and sort by similarity score
    recommendations = recommendations.drop_duplicates(subset=['name']).sort_values(
        by="similarity_score", ascending=False
    )
    
    # Get the final workout plan
    workout_plan = recommendations.head(workout_size)
    
    # Format the plan
    final_plan = workout_plan[["name", "muscle", "equipment", "difficulty", "similarity_score"]]
    
    print("\n=== PERSONALIZED WORKOUT PLAN ===")
    print(f"Goal: {user_preferences['goal']}")
    print(f"Experience: {user_preferences['experience']}")
    print(f"Equipment: {user_preferences['equipment']}")
    print(f"Target muscles: {user_preferences['muscle']}")
    print("\nRECOMMENDED EXERCISES:")
    print(final_plan.to_string(index=False))
else:
    print("No suitable exercises found")

Recommended workout size: 6 exercises

=== PERSONALIZED WORKOUT PLAN ===
Goal: lose weight
Experience: advance
Equipment: machine
Target muscles: back, biceps

RECOMMENDED EXERCISES:
                               name      muscle equipment   difficulty  similarity_score
                       barbell curl      biceps   barbell intermediate          0.514455
                 ez-bar spider curl      biceps   barbell intermediate          0.514455
             wide-grip barbell curl      biceps   barbell     beginner          0.499352
                              rower middle_back   machine intermediate          0.448061
         reverse-grip bent-over row middle_back   barbell intermediate          0.000000
alternating sit-through with crunch middle_back     other intermediate          0.000000
